In [9]:
def FFNN(output_activation,Input_shape,Output_shape,loss_function):
    inputs = []
    predictions = []
    
    #Network structure
    layer1 = Dense(100, activation='relu')
    layer2 = Dense(100, activation='relu')
    layer3 = Dense(100, activation='relu')
    layer4 = Dense(Output_shape, activation=output_activation)
    
    #Loops data across trading days through network
    for i in range(steps):
        sinput = Input(shape=(Input_shape,))
        x = layer1(sinput)
        x = layer2(x)
        x = layer3(x)
        sprediction = layer4(x)
        inputs.append(sinput)
        predictions.append(sprediction)
    
    #Compiles model
    predictions = Concatenate(axis=-1)(predictions)#¤still dont know what this does. Understand it.
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam', loss=loss_function, metrics=[])
    return model

done


In [3]:
#this is not used as the results are bad.
#¤ consider deleting this.
def FFNN_sigmoid_linear(output_activation,Input_shape,Output_shape,loss_function):
    inputs = []
    predictions = []
    
    #Network structure
    layer1 = Dense(100, activation='relu')
    layer2 = Dense(100, activation='relu')
    layer3 = Dense(100, activation='relu')
    layer4 = Dense(Output_shape, activation=output_activation)
    
    layer4_1 = Dense(1,activation='sigmoid')
    layer4_2 = Dense(1,activation='linear')
    
    #Structure of network
    for i in range(steps):
        sinput = Input(shape=(Input_shape,))
        x = layer1(sinput)
        x = layer2(x)
        x = layer3(x)
        pred_1 = layer4_1(x)
        pred_2 = layer4_2(x)
        sprediction = tf.concat([pred_1, pred_2],axis=-1)
        inputs.append(sinput)#appends input for next timestep. such that the network uses all information up to the given timepoint.
        #¤is it recurrent now? Or do we need to add predictions to input. something like inputs.append(spredictions)
        predictions.append(sprediction)#appends output such that we get a vector for all timepoints in the end.
    
    #Builds the model
    predictions = Concatenate(axis=-1)(predictions)#¤still dont know what this does. Understand it.
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam', loss=loss_function, metrics=[])
    return model

In [4]:
#final RNN model
def RNN_final(output_activation,Input_shape,Output_shape,loss_function):
    n_nodes=[100,100,100,Output_shape]
    Activation=['relu','relu','relu',output_activation]
    n_layers=4 #Hidden layer and output layer.
    inputs = Input(shape=(steps,Input_shape))
    cells=[tf.keras.layers.SimpleRNNCell(n_nodes[i],activation=Activation[i]) for i in range(n_layers)]
    outputs = tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells(cells),
                                  stateful=False, 
                                  return_sequences=True, 
                                  return_state=False)(inputs)
    model=Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam',loss=loss_function)
    return model

In [5]:
#this seems to work even better without transaction costs than the RNN function.
#and it is assumed to be closer to what the paper does.
#with the 'relu' and 'linear' specified then it needs to train for more epochs, but it can get to the right model.
#deltas2 are not great though.
    #deltas2 are better without 'relu' (Dense has 'linear' by default)
    #consider not having 'relu', but its not a simple decision. 
def RNN_cell(output_activation,Input_shape,Output_shape,loss_function):
    n_hidden_units=100
    n_hidden_layers=3
    inputs = Input(shape=(steps,Input_shape))
    cells=[tf.keras.layers.SimpleRNNCell(n_hidden_units,activation='relu') for _ in range(n_hidden_layers)]
    outputs = tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells(cells),
                              stateful=False, 
                              return_sequences=True, 
                              return_state=False)(inputs)
    outputs = tf.keras.layers.Dense(Output_shape,activation=output_activation)(outputs)
    model=Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam',loss=loss_function)
    return model

In [6]:
#This works without transaction costs!
#It is however not exactly what the paper does.
def RNN(output_activation,Input_shape,Output_shape,loss_function):
    model_input=Input(shape=(steps,Input_shape))
    x=SimpleRNN(100,return_sequences=True,activation='relu')(model_input)
    x=SimpleRNN(100,return_sequences=True,activation='relu')(x)
    x=SimpleRNN(100,return_sequences=True,activation='relu')(x)
    output=Dense(Output_shape,activation=output_activation)(x)
    model=Model(inputs=model_input,outputs=output)
    model.compile(optimizer='adam',loss=loss_function)
    return model

In [7]:
def RNN_simple(loss_function):
    model_input=Input(shape=(steps,3))
    model_output=SimpleRNN(2,return_sequences=True)(model_input)
    model=Model(inputs=model_input,outputs=model_output)
    model.compile(optimizer='adam', loss=loss_function)
    return model